In [31]:
import pandas as pd
import ast
from torch.nn.utils.rnn import pad_sequence
import os
import ast
from transformers import GPT2Tokenizer
from sklearn.model_selection import train_test_split

In [5]:
#importing the previous preprocessing script's output csv
recipes_data = pd.read_csv('preprocessed_recipes_full.csv')

In [7]:
print(recipes_data.shape)
recipes_data.head()

(13501, 14)


,Unnamed: 0,Ingredients,Instructions,Simplified_Ingredients,Simplified_Instructions,Instr_Tok_Wrds,Ingredients_Tok,Instructions_Tok,Ingr_Len,Instr_Len,Ingr_Tnsr,Instr_Tnsr,Padded_Length_Ingr,Padded_Length_Instr
0,0,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...","['chicken', 'kosher_salt', 'acorn_squash', 'sa...",pat chicken dry with paper towels season all o...,"['pat', 'chicken', 'dry', 'with', 'paper', 'to...","[1002, 7280, 24753, 1322, 40329, 39949, 5605, ...","[1002, 32829, 1009, 45603, 5799, 21078, 22700,...",21,2454,"[1002, 7280, 24753, 1322, 40329, 39949, 5605, ...","[1002, 32829, 1009, 45603, 5799, 21078, 22700,...",53,10565
1,1,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,"['egg_whites', 'new_potatoes', 'teaspoons_kosh...",preheat oven to 400f and line a rimmed baking ...,"['preheat', 'oven', 'to', '400f', 'and', 'line...","[1002, 15973, 28327, 49267, 3879, 39949, 50508...","[1002, 32829, 37882, 15751, 21078, 15751, 1009...",7,592,"[1002, 15973, 28327, 49267, 3879, 39949, 50508...","[1002, 32829, 37882, 15751, 21078, 15751, 1009...",53,10565
2,2,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,"['evaporated_milk', 'milk', 'garlic_powder', '...",place a rack in middle of oven preheat to 400 ...,"['place', 'a', 'rack', 'in', 'middle', 'of', '...","[1002, 16721, 27238, 19315, 29079, 43299, 3879...","[1002, 32829, 24817, 1009, 5799, 15751, 1009, ...",10,832,"[1002, 16721, 27238, 19315, 29079, 43299, 3879...","[1002, 32829, 24817, 1009, 5799, 15751, 1009, ...",53,10565
3,3,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,"['round_italian_loaf_cut_inch_cubes', 'tablesp...",preheat oven to 350f with rack in middle gener...,"['preheat', 'oven', 'to', '350f', 'with', 'rac...","[1002, 40082, 47386, 36793, 5540, 29125, 6812,...","[1002, 32829, 37882, 15751, 21078, 15751, 1009...",13,1637,"[1002, 40082, 47386, 36793, 5540, 29125, 6812,...","[1002, 32829, 37882, 15751, 21078, 15751, 1009...",53,10565
4,4,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,"['dark_brown_sugar', 'hot_water', 'bourbon', '...",stir together brown sugar and hot water in a c...,"['stir', 'together', 'brown', 'sugar', 'and', ...","[1002, 13986, 22592, 4705, 25316, 48768, 19452...","[1002, 40282, 45603, 22700, 37882, 45603, 2856...",6,264,"[1002, 13986, 22592, 4705, 25316, 48768, 19452...","[1002, 40282, 45603, 22700, 37882, 45603, 2856...",53,10565


In [8]:
gpt_data = recipes_data[['Simplified_Ingredients', 'Simplified_Instructions']].copy()
gpt_data.head(1)

,Simplified_Ingredients,Simplified_Instructions
0,"['chicken', 'kosher_salt', 'acorn_squash', 'sa...",pat chicken dry with paper towels season all o...


In [13]:
# Converting ingredients to gpt input by removing underscores and turning into comma
# separated string from string array
def convert_to_gpt_input(input):
  converted = ''
  for ingredient in ast.literal_eval(input):
    ni = ingredient.replace('_', ' ')
    converted += ni + ", "
  return converted

gpt_data['Conv_Ingr'] = gpt_data['Simplified_Ingredients'].apply(convert_to_gpt_input)
gpt_data.head(5)

,Simplified_Ingredients,Simplified_Instructions,Conv_Ingr
0,"['chicken', 'kosher_salt', 'acorn_squash', 'sa...",pat chicken dry with paper towels season all o...,"chicken, kosher salt, acorn squash, sage, rose..."
1,"['egg_whites', 'new_potatoes', 'teaspoons_kosh...",preheat oven to 400f and line a rimmed baking ...,"egg whites, new potatoes, teaspoons kosher sal..."
2,"['evaporated_milk', 'milk', 'garlic_powder', '...",place a rack in middle of oven preheat to 400 ...,"evaporated milk, milk, garlic powder, onion po..."
3,"['round_italian_loaf_cut_inch_cubes', 'tablesp...",preheat oven to 350f with rack in middle gener...,"round italian loaf cut inch cubes, tablespoons..."
4,"['dark_brown_sugar', 'hot_water', 'bourbon', '...",stir together brown sugar and hot water in a c...,"dark brown sugar, hot water, bourbon, lemon ju..."


In [15]:
# Converting from string of comma separated ingredients into prompt structure compatible with gpt models
gpt_data['To_Input'] = "Given these ingredients <INGR> " + gpt_data['Conv_Ingr'] + " The recipe is <RECIPE> " + gpt_data['Simplified_Instructions']
gpt_data['To_Input'][0]

'Given these ingredients <INGR> chicken, kosher salt, acorn squash, sage, rosemary, butter melted room temperature, allspice, crushed red pepper flakes, black pepper, loaf white bread torn pieces, apples cored cut pieces, extravirgin olive oil, red onion thinly sliced, apple cider vinegar, white miso, allpurpose flour, butter room temperature, dry white wine, cups chicken broth, white miso, kosher salt pepper,  The recipe is <RECIPE> pat chicken dry with paper towels season all over with 2 tsp salt and tie legs together with kitchen twine let sit at room temperature 1 hour meanwhile halve squash and scoop out seeds run a vegetable peeler along ridges of squash halves to remove skin cut each half into ½ thick wedges arrange on a rimmed baking sheet combine sage rosemary and 6 tbsp melted butter in a large bowl pour half of mixture over squash on baking sheet sprinkle squash with allspice red pepper flakes and ½ tsp salt and season with black pepper toss to coat add bread apples oil and 

In [20]:

gpt_data.isna().sum()

Simplified_Ingredients     0
Simplified_Instructions    8
Conv_Ingr                  0
To_Input                   8
dtype: int64

In [22]:
# Dropping null entries
gpt_data = gpt_data.dropna()
print(gpt_data.shape)

(13493, 4)


In [ ]:
# Using gpt2 pretrained tokenizer to tokenize text inputs, and remove any that exceed
# the max token length of 1024
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt_data['Tokenized'] = gpt_data['To_Input'].apply(lambda x: tokenizer.encode(x, truncation = False))
gpt_data['Tok_Len'] = gpt_data['Tokenized'].apply(len)
gpt_data = gpt_data[gpt_data['Tok_Len'] <= 1024]
print(gpt_data.shape)

Token indices sequence length is longer than the specified maximum sequence length for this model (1283 > 1024). Running this sequence through the model will result in indexing errors


(13457, 6)


In [27]:
gpt_data.head()

,Simplified_Ingredients,Simplified_Instructions,Conv_Ingr,To_Input,Tokenized,Tok_Len
0,"['chicken', 'kosher_salt', 'acorn_squash', 'sa...",pat chicken dry with paper towels season all o...,"chicken, kosher salt, acorn squash, sage, rose...","Given these ingredients <INGR> chicken, kosher...","[15056, 777, 9391, 1279, 2751, 49, 29, 9015, 1...",576
1,"['egg_whites', 'new_potatoes', 'teaspoons_kosh...",preheat oven to 400f and line a rimmed baking ...,"egg whites, new potatoes, teaspoons kosher sal...","Given these ingredients <INGR> egg whites, new...","[15056, 777, 9391, 1279, 2751, 49, 29, 5935, 1...",157
2,"['evaporated_milk', 'milk', 'garlic_powder', '...",place a rack in middle of oven preheat to 400 ...,"evaporated milk, milk, garlic powder, onion po...",Given these ingredients <INGR> evaporated milk...,"[15056, 777, 9391, 1279, 2751, 49, 29, 28959, ...",229
3,"['round_italian_loaf_cut_inch_cubes', 'tablesp...",preheat oven to 350f with rack in middle gener...,"round italian loaf cut inch cubes, tablespoons...",Given these ingredients <INGR> round italian l...,"[15056, 777, 9391, 1279, 2751, 49, 29, 2835, 3...",392
4,"['dark_brown_sugar', 'hot_water', 'bourbon', '...",stir together brown sugar and hot water in a c...,"dark brown sugar, hot water, bourbon, lemon ju...",Given these ingredients <INGR> dark brown suga...,"[15056, 777, 9391, 1279, 2751, 49, 29, 3223, 7...",89


In [29]:
# Generating train and test splits
final_inputs = gpt_data['To_Input'].copy()
gpt_train, gpt_rest = train_test_split(final_inputs, test_size = 0.8, train_size = 0.2, random_state = 4701)
gpt_test, gpt_val = train_test_split(gpt_rest, train_size = 0.5, test_size = 0.5, random_state = 4701)

In [ ]:
# Outputting the splits to CSVs
gpt_train.to_csv('./GPT_CSVs/gpt_train.csv')
gpt_test.to_csv('./GPT_CSVs/gpt_test.csv')
gpt_val.to_csv('./GPT_CSVs/gpt_val.csv')